Polynomials turn out to be critically important for modern cryptography. You might not have seen polynomials in a long time, so we'll review the basic concepts of polynomials from first principles. First, recall that a "function" is a rule for transforming an input from one representation to another. The concept of "function" is pretty similar in both computer science and mathematics, so let's look at some simple examples of functions in Python.

In [1]:
def f(x):
  return x + 1
print(f(1))
print(f(2))

2
3


This seems pretty simple right? How can we write `f` in mathematical notation? It's pretty simple too
$$ f(x) = x + 1 $$
Ok, let's look at some more complicated functions.

In [3]:
def g(x):
  return x**3 + x**2 + x - 1
print(g(5))

154


Mathematically, we would write `g` as a function as follows
$$g(x) = x^3 + x^2 + x - 1$$
Note that there's something simple about the function `f` and `g`. In particular, they only use a few simple operations like addition, subtraction, and multiplication. (Exponentiation is just repeated multiplication). More generally, any function that uses these simple operations is called a "polynomial." Let's take a look at a few examples of polynomials
$$h(x) = x^2$$
$$\ell(x) = (x + 7)^4 + (x-2)^2 - x$$
Alright, this makes sense. Polynomials are a simple class of function that accepts in numbers and computes operations on them. But why should we care? Well, things get interesting we we start to combine the notion of polynomials with the ideas about finite fields that we learned in the previous tutorial.

So why are polynomials interesting for computer science? The key concept is that of "arithmetization." A core idea is that any computation can be transformed into a a polynomial. Why does this make sense? Well, first let's talk about digital circuits. Any computation can be encoded as a "digital circuit." What's a digital circuit you ask? Well, you can think of it as a function built up from boolean primitives such as AND, OR, NOT and the like.

![digital_gates](https://www.oreilly.com/library/view/introduction-to-digital/9780470900550/images/ch003-f002.jpg)

These gates can be wired into one another to form a circuit. 

![digital_circuit](http://www.edwardbosworth.com/My5155_Slides/Chapter03/DigitalCircuitsAndBooleanExpressions_files/image002.jpg)

It takes a little imagining, but any algorithm can be transformed into a digital circuit. Why does this make sense? Well just reasoning from first principles we know that all algorithms we write are run on physical chips. Most chips themselves are simply large digital circuits that have the capacity to process "instructions"

![mips_circuit](https://i.imgur.com/6R3Xz.png)

In fact it's well known in computer science that you can transform back and forth between Turing Machines and Circuits as models of computation so we're on very solid ground here. But what does this have to do with polynomials? Well we explained earlier that polynomials are made up of compositions of simple operations. So are circuits. The connection comes through the idea of "arithmetic circuits"

![arithmetic_circuit](https://upload.wikimedia.org/wikipedia/commons/thumb/6/64/ArithmeticCircuit.svg/527px-ArithmeticCircuit.svg.png)

An arithmetic circuit is simply a circuit that uses addition, subtraction, and multiplication instead of boolean operators. In addition, the input values to the circuit are numbers and not booleans. (For our applications, these numbers will come from finite fields).